In [1]:
import numpy as np
import tifffile
from scipy import ndimage
from skimage.filters import gaussian
from skimage.filters import threshold_li
from skimage.measure import label
import os
import glob
from google.colab import drive

def correct_illumination(img, sigma=50):
    """
    Correct uneven illumination using division by background estimation.

    Parameters:
    -----------
    img : ndarray
        Input image
    sigma : float
        Standard deviation for the Gaussian filter used to estimate background

    Returns:
    --------
    illumination_corrected : ndarray
        Illumination-corrected image normalized to [0, 1]
    """
    # Normalize input image to [0, 1] range
    img_min = np.min(img)
    img_max = np.max(img)

    # Check to prevent division by zero
    if img_max > img_min:
        img_norm = (img - img_min) / (img_max - img_min)
    else:
        img_norm = np.zeros_like(img, dtype=float)

    # Apply Gaussian filter to estimate background
    background = ndimage.gaussian_filter(img_norm, sigma=sigma)

    # Use DIVISION instead of subtraction to correct illumination
    illumination_corrected = img_norm / (background + 1e-6)  # Adding small value to prevent division by zero

    # Normalize the result to [0, 1] range
    illumination_min = np.min(illumination_corrected)
    illumination_max = np.max(illumination_corrected)
    if illumination_max > illumination_min:  # Check to prevent division by zero
        illumination_corrected = (illumination_corrected - illumination_min) / (illumination_max - illumination_min)

    return illumination_corrected

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/flow3-x20/Cadherins/tophat'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Membrane_raw'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# No intermediate directory needed - we're not saving the corrected images

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # Extract Cadherins channel based on image format
    if len(img.shape) == 2:
        # Single channel image (already Cadherins)
        print("  Detected single-channel image, using as is")
        cadherins_channel = img
    elif len(img.shape) == 3 and img.shape[0] == 3:
        # Format is (C, H, W)
        print("  Detected format: (C, H, W)")
        cadherins_channel = img[1]  # Second channel - assuming Cadherins is in channel 1
                                    # Adjust this index if your Cadherins are in a different channel
    elif len(img.shape) == 3 and img.shape[2] == 3:
        # Format is (H, W, C)
        print("  Detected format: (H, W, C)")
        cadherins_channel = img[:, :, 1]  # Second channel - assuming Cadherins is in channel 1
                                          # Adjust this index if your Cadherins are in a different channel
    else:
        print(f"  Unexpected image shape: {img.shape}. Using first channel/plane.")
        if len(img.shape) == 3:
            cadherins_channel = img[0] if img.shape[0] < img.shape[1] else img[:, :, 0]
        else:
            cadherins_channel = img

    # Apply illumination correction
    print("  Applying illumination correction...")
    cadherins_corrected = correct_illumination(cadherins_channel, sigma=50)

    # No need to save intermediate illumination-corrected image

    # Apply Gaussian blur to reduce noise on the corrected image
    print("  Applying Gaussian blur...")
    cadherins_smoothed = gaussian(cadherins_corrected, sigma=1)

    # Apply Li thresholding
    print("  Performing Li thresholding...")
    thresh = threshold_li(cadherins_smoothed)
    binary = cadherins_smoothed > thresh

    # Save the binary mask (main output)
    mask_output = os.path.join(output_dir, f"{base_name}_cadherins_mask.tif")
    tifffile.imwrite(mask_output, binary.astype(np.uint8) * 255)
    print(f"  Saved binary mask to {mask_output}")

    # Print some statistics about the segmentation
    print(f"  Threshold value used: {thresh}")
    print(f"  Processing complete for {filename}\n")

print("All processing complete!")

Mounted at /content/drive
Found 8 .tif files to process
Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Nuclei_regional_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying illumination correction...
  Applying Gaussian blur...
  Performing Li thresholding...
  Saved binary mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Membrane_raw/denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Nuclei_regional_tophat_cadherins_mask.tif
  Threshold value used: 0.05379337430168971
  Processing complete for denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Nuclei_regional_tophat.tif

Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_Nuclei_regional_tophat.tif
  Image shape: (1024, 1024)
  Detected single-channel image, using as is
  Applying illumination correction...
  Applying Gaussian blur...
  Performing Li thresholding...
  Saved binary mask to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flo

In [5]:
import numpy as np
import tifffile
import os
import glob
import re
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
cadherins_input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Membrane_raw'
golgi_input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Nuclei'  # Golgi directory
cadherins_output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Membrane'

# Create output directory if it doesn't exist
os.makedirs(cadherins_output_dir, exist_ok=True)

# Find all Cadherins mask files
cadherins_files = glob.glob(os.path.join(cadherins_input_dir, '*.tif'))
print(f"Found {len(cadherins_files)} Cadherins mask files")

# Get all Golgi mask files
golgi_files = glob.glob(os.path.join(golgi_input_dir, '*.tif'))
print(f"Found {len(golgi_files)} Golgi mask files")

# Define a regex pattern to extract the consistent part of the filename
# This pattern looks for the format: denoised_<pressure>Pa_U_<date>_<magnification>_<location>_Flat_seq<number>
pattern = r'(denoised_[\d.]+Pa_U_\d+[a-z]+\d+_\d+x_[A-Za-z\d]+_Flat_seq\d+)'

# Print all Golgi filenames for debugging
print("\nList of all Golgi files:")
for golgi_file in golgi_files:
    print(f"  - {os.path.basename(golgi_file)}")

# Process each Cadherins mask
for cadherins_file in cadherins_files:
    # Get filename
    cadherins_filename = os.path.basename(cadherins_file)
    print(f"\nProcessing: {cadherins_filename}")

    # Extract common part from filename using regex
    match = re.search(pattern, cadherins_filename)
    if match:
        common_part = match.group(1)
    else:
        # If pattern doesn't match, use the original approach
        if '_cadherins_mask' in cadherins_filename.lower():
            common_part = cadherins_filename.split('_cadherins_mask')[0]
        else:
            # Fallback to just removing extension if pattern doesn't match
            common_part = os.path.splitext(cadherins_filename)[0]

    print(f"  Common identifier: {common_part}")

    # Load the Cadherins mask
    cadherins_mask = tifffile.imread(cadherins_file)
    cadherins_binary = cadherins_mask > 0  # Convert to binary boolean mask

    # Look for matching Golgi file
    matching_golgi_file = None

    for golgi_file in golgi_files:
        golgi_filename = os.path.basename(golgi_file)
        # Check if the common part is in the Golgi filename
        if common_part in golgi_filename:
            # Further check if this is really a Golgi file (could have "_Golgi_" or "golgi" in name)
            if '_golgi_' in golgi_filename.lower() or 'golgi' in golgi_filename.lower():
                matching_golgi_file = golgi_file
                break

    # If no match found with the strict criteria, try a more lenient approach
    if not matching_golgi_file:
        for golgi_file in golgi_files:
            golgi_filename = os.path.basename(golgi_file)
            # Try matching just based on the sequence number and other key identifiers
            seq_match = re.search(r'seq(\d+)', cadherins_filename)
            if seq_match and seq_match.group(0) in golgi_filename:
                # Check for other identifiers like pressure value and date
                pressure_match = re.search(r'([\d.]+Pa)', cadherins_filename)
                if pressure_match and pressure_match.group(0) in golgi_filename:
                    matching_golgi_file = golgi_file
                    print(f"  Found match using sequence number and pressure: {os.path.basename(golgi_file)}")
                    break

    # Extract the original filename without extension for output naming
    original_name_no_ext = os.path.splitext(cadherins_filename)[0]

    # Process and save cleaned mask if a matching file was found
    if matching_golgi_file:
        # Load the Golgi mask
        golgi_mask = tifffile.imread(matching_golgi_file)
        golgi_binary = golgi_mask > 0  # Convert to binary boolean mask

        # Calculate cleaned mask (Cadherins without Golgi overlap)
        cleaned_cadherins = cadherins_binary & ~golgi_binary

        print(f"  Removed Golgi overlap from Cadherins mask using {os.path.basename(matching_golgi_file)}")

        # Calculate statistics
        original_pixels = np.sum(cadherins_binary)
        cleaned_pixels = np.sum(cleaned_cadherins)
        removed_pixels = original_pixels - cleaned_pixels
        removed_percent = (removed_pixels / original_pixels * 100) if original_pixels > 0 else 0
        print(f"  Removed {removed_pixels} pixels ({removed_percent:.2f}% of original)")

        # Save the cleaned binary mask
        cleaned_file = os.path.join(cadherins_output_dir, f"{original_name_no_ext}_cleaned.tif")
        tifffile.imwrite(cleaned_file, cleaned_cadherins.astype(np.uint8) * 255)
        print(f"  Saved cleaned mask to {cleaned_file}")

    else:
        print(f"  Warning: No matching Golgi mask found for common identifier: {common_part}")

        # Save the original binary mask if no matching Golgi file was found
        original_file = os.path.join(cadherins_output_dir, f"{original_name_no_ext}_original.tif")
        tifffile.imwrite(original_file, cadherins_binary.astype(np.uint8) * 255)
        print(f"  No cleaning performed. Saved original mask to {original_file}")

print("\nAll processing complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 8 Cadherins mask files
Found 8 Golgi mask files

List of all Golgi files:
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_filtered_mask.tif
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_Cadherins_filtered_mask.tif
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq003_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq001_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq002_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq003_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq004_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask.tif

Processing: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Nuclei_regional_tophat_cadherins_mask.tif
  Common identifier: denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001
  Found match 